会津若松市の公用車走行のOpenDataで、路面の凹凸箇所をGoogle Mapにプロットしてみる。

In [2]:
from math import sqrt, cos, sin, asin, radians, atan2
import json
import requests
import gmaps
import numpy as np

まず、公用車の走行データをData4Citizienより取得。1回の上限が1万件なので、offsetをつかって10回に分けてデータを取得する(10万件)。データは、100ms周期の3軸加速度が取得できる。

In [3]:
def getCarDatasets():
  LIMIT = 10000
  SIZE = 100000
  
  for i in xrange(SIZE/LIMIT):
    pos_offset = LIMIT * (i)
    url = 'http://www.data4citizen.jp/app/developer/query/query?odql=SELECT%20car_name%0A%2C%20latitude%0A%2C%20longitude%0A%2C%20gps_error_meter%0A%2C%20accel_x_transverse%0A%2C%20accel_y_longitudinal%0A%2C%20accel_z_vertical%0AFROM%20O_CAR_TRAFFIC_DATA%20limit%20'+str(LIMIT)+'%20offset%20'+str(pos_offset)
    r = requests.get(url)
    if i == 0:
      data = r.json()["data"]
    else:
      data.extend(r.json()["data"])

  return data

会津若松市の公用車データ10万件のデータを取得。

In [4]:
carDatasets = getCarDatasets()
len(carDatasets)

100000

信号機データを取得。

In [5]:
def accident(before_axis, latest_axis):
  ACCIDENT = 10000
  if np.power((before_axis*100-latest_axis*100),2) > ACCIDENT:
    return True
  else:
    return False
  
accidentPoints = np.empty((0,2), float)
(x, y, z) = (0, 0, 0)
for item in carDatasets:
  lat = item["latitude"]
  lon = item["longitude"]
  (_x, _y, _z) = (x, y, z)
  x = item["accel_x_transverse"]
  y = item["accel_y_longitudinal"]
  z = item["accel_z_vertical"]
  if accident(_z, z):
    accidentPoints = np.append(accidentPoints, np.array([[lat, lon]]), axis=0)

len(accidentPoints)

383

最後にGoogle Map上にPlot。

In [18]:
gmaps.configure(api_key="AIza...")

starbucks_layer = gmaps.symbol_layer(
    accidentPoints.tolist(), fill_color="red", stroke_color="red", scale=1
)

m = gmaps.Map()
m.add_layer(starbucks_layer)
m

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
